# Patch Discriminator

In [ ]:
import torch.nn as nn

In [ ]:
class NLayerDiscriminator(nn.Module):
    """
    'PatchGAN' classifier described in the original pix2pix paper.
    It can classify whether 70×70 overlapping patches are real or fake.
    Such a patch-level discriminator architecture has fewer parameters
    than a full-image discriminator and can work on arbitrarily-sized images
    in a fully convolutional fashion.
    """
    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.InstanceNorm2d):
        """
        Construct a PatchGAN discriminator
        
        Parameters:
            1. input_nc (int): the number of channels in input images
            2. ndf (int): the number filters
            3. n_layers (int): the number of conv layers in the discriminator
            4. norm_layer (nn.Module): normalization layer 
        """
        super(NLayerDiscriminator, self).__init__()
        use_bias = (norm_layer == nn.InstanceNorm2d)
    
        kernel_size = 4
        padding = 1
        
        # Leading Conv
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=kernel_size, stride=2, padding=padding),
            nn.LeakyReLU(.2, True)
        ]
        
        # gradually increase the number of filters and decrease the dimension of the image
        mult_i = 1
        for n in range(n_layers):
            mult_o = min(mult_i * 2, 8)
            ic, oc = ndf * mult_i, ndf * mult_o
            sequence += [
                nn.Conv2d(
                    ic, oc, 
                    kernel_size=kernel_size, 
                    stride=2, 
                    padding=padding, 
                    bias=use_biase),
                norm_layer(oc),
                nn.LeakyReLU(.2, True)
            ]
            mult_i = mult_o
        
        
        mult_o = min(mult_i * 2, 8)
        ic, oc = ndf * mult_i, ndf * mult_o
        sequence += [
            nn.Conv2d(ic, oc, kernel_size=kernel_size, stride=1, padding=padding, bias=use_biase),
            norm_layer(oc),
            nn.LeakyReLU(.2, True)
        ]
        
        sequence += [nn.Conv2d(oc, 1, kernel_size=kernel_size, stride=1, padding=padding)]
        
        self.model = nn.Sequential(*sequence)
        
    def forward(self, x):
        return self.model(x)